# Data science with xarray

Hello and welcome to your intro to `xarray` for data science as part of NCI's Parallel Python data science course.

This notebook is designed to cover the fundamentals of xarray, highlighting some similarities with tools you have already been introduced too such as `numpy` and `CuPy` and some of the advantages of using `xarray`.


First of all what is `xarray`? `xarray` is a project that came out of climate and geophysics research, recognising the need for rapid, scalable and easily manipulated N dimensional array data **with labels and metadata**. 

But wait, doesn't `numpy` provide an N Dimensional array? Yes it does, but `xarray` provides the ability to use labelled and metadata rich arrays, providing to quote the manual a " more intuitive, more concise, and less error-prone developer experience"

This is done by providing a label based API that simplifies a lot of the manual book keeping of working with `numpy` arrays directly.

No longer will you forget what experiment this data came from, what that tensor dimension was, or what array position corresponded to with time point! Even better these, augmented N dimensional arrays can be combined to make massive datasets, enabling analysis of huge volumes of data. It's all there in  the one datastructure that can be easily stored and shared, as is done on a massive scale using `xarray` in the climate modelling, geophysics and astrophysics communities to name just a few. 


Alright, lets jump in!

In [2]:
# lets import xarray and numpy
import xarray as xr
import numpy as np

The `DataArray` is the `xarray` equivalent of the `numpy` `ndarray` and will be the first focus of our intro. Lets make a 2 x 3 DataArray using a numpy array, so we can get the hang of how to work with it. 

In [9]:
inp = np.arange(6).reshape(2,3)
data = xr.DataArray(inp, dims=("x", "y"))
data

<xarray.DataArray (x: 2, y: 3)>
array([[0, 1, 2],
       [3, 4, 5]])
Dimensions without coordinates: x, y

We can see that our data array looks a lot like a numpy array, but with two labelled dimensions `x` and `y`. We can access our values directly using the `values` attribute as shown below. 

In [14]:
data.values

array([[0, 1, 2],
       [3, 4, 5]])

We can also access our dimensions using the `dims` attribute as shown below

In [15]:
data.dims

('x', 'y')

The eagle eyed amongst you may have  spotted the `coords` attribute. This is used to associate a value along a particular axis with another value, that could for example correspond to the time or location it was measured. The possibilities are only limited by your imagination

We set our `coords` using a dictionary corresponding to the `coords` along a particular axis.

In [23]:
coords = {"x":[10,20], "y":[0.1, 0.2, 0.3]}
data = data.assign_coords(coords)

**Notice the detail above**, we had to assign our `DataArray` to a new object (in this case we overwrote `data`) for our change to persist. This pattern will be familiar to those who use `pandas`.

Okay cool, how can we access the data in our array? We can use four kinds of slicing. 

* Numpy like using integer locations
* using `loc` like in pandas
* using an integer select (`isel`), combining a dimension name and integer label
* using a selection (`sel`) based on coordinate combining a dimension name and coordinate value

The following selections all give the same set of values, those at `x=10`:

Numpy like using array indexing:

In [27]:
data[0, :]

<xarray.DataArray (y: 3)>
array([0, 1, 2])
Coordinates:
    x        int64 10
  * y        (y) float64 0.1 0.2 0.3

Pandas like using `loc`

In [28]:
data.loc[10]

<xarray.DataArray (y: 3)>
array([0, 1, 2])
Coordinates:
    x        int64 10
  * y        (y) float64 0.1 0.2 0.3

An xarray integer selection

In [31]:
data.isel(x=0)

<xarray.DataArray (y: 3)>
array([0, 1, 2])
Coordinates:
    x        int64 10
  * y        (y) float64 0.1 0.2 0.3

An xarray selection

In [32]:
data.sel(x=10)

<xarray.DataArray (y: 3)>
array([0, 1, 2])
Coordinates:
    x        int64 10
  * y        (y) float64 0.1 0.2 0.3

When dealing with a complicated multidimensional dataset, I and the creators of `xarray` would argue that the last two are simple and powerful

In [33]:
data.attrs["long_name"] = "random velocity"
data.attrs["units"] = "metres/sec"
data.attrs["description"] = "A random variable created as an example."


In [34]:
data.x.attrs["units"] = "x units"
data.x

<xarray.DataArray 'x' (x: 2)>
array([10, 20])
Coordinates:
  * x        (x) int64 10 20
Attributes:
    units:    x units

In [12]:
data += 100
data

<xarray.DataArray (x: 2, y: 3)>
array([[100.01804065,  99.02779552, 101.17313936],
       [ 99.71407696,  98.65829773, 100.27335784]])
Coordinates:
  * x        (x) int64 10 20
Dimensions without coordinates: y
Attributes:
    long_name:    random velocity
    units:        metres/sec
    description:  A random variable created as an example.

In [13]:
data.T


<xarray.DataArray (y: 3, x: 2)>
array([[100.01804065,  99.71407696],
       [ 99.02779552,  98.65829773],
       [101.17313936, 100.27335784]])
Coordinates:
  * x        (x) int64 10 20
Dimensions without coordinates: y
Attributes:
    long_name:    random velocity
    units:        metres/sec
    description:  A random variable created as an example.

In [14]:
a = xr.DataArray(np.random.randn(3), [data.coords["y"]])
a

<xarray.DataArray (y: 3)>
array([ 0.53137388, -0.0268387 , -0.24259061])
Coordinates:
  * y        (y) int64 0 1 2

In [15]:
b = xr.DataArray(np.random.randn(4), dims="z")
b

<xarray.DataArray (z: 4)>
array([-1.26279468, -1.1851513 , -1.82177888,  1.11401864])
Dimensions without coordinates: z

In [16]:
c = a + b
c

<xarray.DataArray (y: 3, z: 4)>
array([[-0.7314208 , -0.65377743, -1.290405  ,  1.64539252],
       [-1.28963338, -1.21199   , -1.84861757,  1.08717994],
       [-1.50538529, -1.42774192, -2.06436949,  0.87142802]])
Coordinates:
  * y        (y) int64 0 1 2
Dimensions without coordinates: z

In [18]:
c

<xarray.DataArray (y: 3, z: 4)>
array([[-0.7314208 , -0.65377743, -1.290405  ,  1.64539252],
       [-1.28963338, -1.21199   , -1.84861757,  1.08717994],
       [-1.50538529, -1.42774192, -2.06436949,  0.87142802]])
Coordinates:
  * y        (y) int64 0 1 2
Dimensions without coordinates: z

In [42]:
x = xr.DataArray(np.arange(10), dims="x", coords= {"x": np.arange(10)/10})

y = xr.DataArray(np.arange(3), dims="y", coords = {"y": np.arange(3)/10})

In [43]:
x

<xarray.DataArray (x: 10)>
array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])
Coordinates:
  * x        (x) float64 0.0 0.1 0.2 0.3 0.4 0.5 0.6 0.7 0.8 0.9

In [44]:
y

<xarray.DataArray (y: 3)>
array([0, 1, 2])
Coordinates:
  * y        (y) float64 0.0 0.1 0.2

In [45]:
z = x*y
z

<xarray.DataArray (x: 10, y: 3)>
array([[ 0,  0,  0],
       [ 0,  1,  2],
       [ 0,  2,  4],
       [ 0,  3,  6],
       [ 0,  4,  8],
       [ 0,  5, 10],
       [ 0,  6, 12],
       [ 0,  7, 14],
       [ 0,  8, 16],
       [ 0,  9, 18]])
Coordinates:
  * x        (x) float64 0.0 0.1 0.2 0.3 0.4 0.5 0.6 0.7 0.8 0.9
  * y        (y) float64 0.0 0.1 0.2

In [47]:
z[1,1]

<xarray.DataArray ()>
array(1)
Coordinates:
    x        float64 0.1
    y        float64 0.1

In [48]:
w = xr.DataArray(np.arange(10), dims="x", coords= {"x": np.arange(10)/10})

In [50]:
q = w*x
q

<xarray.DataArray (x: 10)>
array([ 0,  1,  4,  9, 16, 25, 36, 49, 64, 81])
Coordinates:
  * x        (x) float64 0.0 0.1 0.2 0.3 0.4 0.5 0.6 0.7 0.8 0.9

In [53]:
l = z * w
l

<xarray.DataArray (x: 10, y: 3)>
array([[  0,   0,   0],
       [  0,   1,   2],
       [  0,   4,   8],
       [  0,   9,  18],
       [  0,  16,  32],
       [  0,  25,  50],
       [  0,  36,  72],
       [  0,  49,  98],
       [  0,  64, 128],
       [  0,  81, 162]])
Coordinates:
  * x        (x) float64 0.0 0.1 0.2 0.3 0.4 0.5 0.6 0.7 0.8 0.9
  * y        (y) float64 0.0 0.1 0.2

In [ ]:
x.save_n